#01.SETUP

In [4]:
!pip install transformers==4.25.1
!pip install bitsandbytes-cuda111==0.26.0
!pip install datasets==1.16.1
!pip install bitsandbytes loguru
!pip install accelerate
!pip install deep_translator
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
ERROR: Could not find a version that satisfies the requirement bitsandbytes-cuda111==0.26.0 (from versions: 0.26.0.post2)
ERROR: No matching distribution found for bitsandbytes-cuda111==0.26.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 

In [5]:
from loguru import logger
import transformers
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise
from tqdm.auto import tqdm
from datasets import load_dataset
from bitsandbytes.optim import Adam8bit
import time, os

# ---------------------> Converting the model to 8 bits <------------------- #
"""
We convert EleutherAI's GPT-J-6B model to 8 bits using facebook's [bitsandbytes](https://github.com/facebookresearch/bitsandbytes) library.
This reduces the model's size from 20Gb down to just 6Gb.
Note that we don't convert linear layer biases to 8 bit as they take up less that 1% of the model's weight anyway.
"""

class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias

    # def forward(self, input):
    #     output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
    #     if self.adapter:
    #         output += self.adapter(input)
    #     return output
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output_cloned = torch.clone(output + self.adapter(input))
            return output_cloned
        else:
            return output

    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"



class DequantizeAndLinear(torch.autograd.Function):
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)

    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias


class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None

    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output

    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"

def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)

    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)


def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr(
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)

#02.LOAD MARYGPT

In [7]:
transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

# ---------------------> Loading EleutherAI/gpt-j-6B config and tokenizer <------------------- #
# config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6b")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6b")

# ---------------------> Downloading gpt-j-6B-8bit model from huggingface <------------------- #
#gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)

# ----------------> Saving gpt-j-6B-8bit model to server <-----------------#
#save_dir = "/home/paperspace/project/saved_models_gpt-j-6B-8bit/gpt-j-6B"
#gpt.save_pretrained(save_dir)
#logger.info("Saved model to {}".format(save_dir))

# ---------------------> Loading saved gpt-j-6B-8bit model <------------------- #
#gpt = GPTJForCausalLM.from_pretrained("./saved_models_gpt-j-6B-8bit/gpt-j-6B",low_cpu_mem_usage=True)
gpt = GPTJForCausalLM.from_pretrained("obake2ai/MaryGPT", device_map="auto", low_cpu_mem_usage=True)
config = transformers.GPTJConfig.from_pretrained("obake2ai/MaryGPT")


device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt.to(device)

# # ---------------------> Text generation example <------------------- #
# prompt = tokenizer("A cat sat on a mat", return_tensors='pt')
# prompt = {key: value.to(device) for key, value in prompt.items()}
# out = gpt.generate(**prompt, min_length=128, max_length=128, do_sample=True)
# logger.info("Generated text: {}".format(tokenizer.decode(out[0])))

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

Some weights of the model checkpoint at obake2ai/MaryGPT were not used when initializing GPTJForCausalLM: ['transformer.h.5.mlp.fc_in.adapter.1.weight', 'transformer.h.15.attn.v_proj.adapter.0.weight', 'transformer.h.18.mlp.fc_out.adapter.0.weight', 'transformer.h.4.mlp.fc_in.adapter.0.weight', 'transformer.h.22.attn.v_proj.adapter.1.weight', 'transformer.h.21.attn.out_proj.adapter.1.weight', 'transformer.h.14.attn.q_proj.adapter.1.weight', 'transformer.h.2.attn.q_proj.adapter.0.weight', 'transformer.h.0.attn.k_proj.adapter.1.weight', 'transformer.h.14.attn.k_proj.adapter.1.weight', 'transformer.h.7.mlp.fc_in.adapter.0.weight', 'transformer.h.26.mlp.fc_out.adapter.1.weight', 'transformer.h.24.attn.q_proj.adapter.1.weight', 'transformer.h.20.mlp.fc_out.adapter.1.weight', 'transformer.h.5.attn.out_proj.adapter.1.weight', 'transformer.h.5.attn.q_proj.adapter.1.weight', 'transformer.h.15.attn.k_proj.adapter.0.weight', 'transformer.h.25.mlp.fc_out.adapter.0.weight', 'transformer.h.22.attn.o

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): FrozenBNBLinear(4096, 50400)
)

# 03.ASK QUESTIONS

In [8]:
import os
import time
import datetime
import requests
import pytz
import random
from deep_translator import GoogleTranslator
from langdetect import detect
import re
import shutil

path_save_dir = "./log"

def modify_text(text):
    sentences = re.findall(r'.+?[.!?]', text)
    if sentences:
        modified_text = ' '.join(sentences)
    else:
        modified_text = text

    modified_text = re.sub(r'\n{2,}', '\n', modified_text)
    modified_text = remove_header(modified_text)

    return modified_text


def translate_to_japanese(text):
    return GoogleTranslator(source='en', target='ja').translate(text).replace("岸優馬", "岸裕真")

def translate_to_english(text):
    return GoogleTranslator(source='ja', target='en').translate(text).replace("岸優馬", "岸裕真")

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False
import random

def is_japanese(text):
    try:
        return detect(text) == 'ja'
    except:
        return False
import random

def remove_header(text):
  return text.replace(question_header, "")

question = "展示のコンセプトを考えて" #@param {type:"string"}
min_words = 60 #@param {type:"number"}
max_words = 120 #@param {type:"number"}

question_header = """
You are MaryGPT, an open-source LLM model fine-tuned on the Gothic novel Frankenstein; or, The Modern Prometheus by Mary Shelley, and an excellent art curator.
"""

print_jp = False
if is_japanese(question):
    question = translate_to_english(question)
    print_jp = True

question_format = f"""
{question_header}

Question: {question}
Answer:
"""

def get_mary_response():
    text_here = question_format
    prompt = tokenizer(text_here, return_tensors='pt')
    prompt = {key: value.to(device) for key, value in prompt.items()}
    out = gpt.generate(**prompt, min_length=min_words, max_length=max_words, do_sample=True)
    text = tokenizer.decode(out[0])[len(question_format):]
    return modify_text(text)

def create_mary_log():
    tz_tokyo = pytz.timezone('Asia/Tokyo')
    current_time = datetime.datetime.now(tz_tokyo)
    formatted_time = current_time.strftime('%Y/%m/%d %H:%M')

    filename = f"log_{current_time.strftime('%Y%m%d_%H%M%S')}.txt"
    with open(os.path.join(path_save_dir, filename), 'w') as file:

        mary_text = get_mary_response()
        if is_english(mary_text):
            translated_text = translate_to_japanese(mary_text)
            file.write(f"\n{translated_text}\n\n")

        file.write(f"{mary_text}\n")
        print(f"{mary_text}\n")
        #file.write(f"***generated: {formatted_time}***\n")

if not os.path.exists(path_save_dir):
    os.makedirs(path_save_dir)

mary_text_en = get_mary_response()
mary_text_jp = translate_to_japanese(mary_text_en)

print("\n thinking...")

if print_jp:
    print(mary_text_jp)
else:
    print(mary_text_en)

# create_mary_log()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 thinking...
最近、展覧会のコンセプトの性質と、それがアーティストの展覧会の実践とより広い意味でどのように関係しているかについての記事が多くあります。この質問は、かなり明白に聞こえると感じるかもしれませんが、そうではありません。展覧会のコンセプトについてどう思いますか? それを見たときに、何が見えますか?
